### This notebook shows examples of how to use the cyclops.processor API on GEMINI.

## Get all patient encounters including ER data, with diagnoses from St. Michael's hospital from March 1 2020 to March 15 2020, with vitals and labs.

In [ ]:
from cyclops.processors.column_names import AGE, DIAGNOSIS_CODE, ENCOUNTER_ID, SEX
from cyclops.processors.constants import SMH
from cyclops.processors.utils import gather_columns
from cyclops.query import gemini

patients = gemini.patients(
    hospitals=[SMH], from_date="2020-03-01", to_date="2020-03-15", include_er_data=True
)
patients_diagnoses = gemini.diagnoses(patients=patients)
patients_vitals = gemini.events(patients=patients, category="vitals")
patients_labs = gemini.events(patients=patients, category="labs")
diagnoses_data = patients_diagnoses.run()
vitals_data = patients_vitals.run()
labs_data = patients_labs.run()

print(f"{len(diagnoses_data)} diagnoses rows extracted!")
print(f"{len(vitals_data)} vitals rows extracted!")
print(f"{len(labs_data)} labs rows extracted!")

patients_diagnoses.save(".", "diagnoses")
patients_vitals.save(".", "vitals")
patients_labs.save(".", "labs")

2022-04-19 09:59:21,968 INFO cyclops.processors.utils - Log file is /mnt/nfs/home/krishnanam/vector-delirium/log.log
2022-04-19 09:59:22,150 INFO cyclops.config  - Log file is /mnt/nfs/home/krishnanam/vector-delirium/log.log
2022-04-19 09:59:22,234 INFO cyclops.query.utils - Log file is /mnt/nfs/home/krishnanam/vector-delirium/log.log
2022-04-19 09:59:22,241 INFO cyclops.utils.profile - Log file is /mnt/nfs/home/krishnanam/vector-delirium/log.log
2022-04-19 09:59:22,246 INFO cyclops.orm     - Log file is /mnt/nfs/home/krishnanam/vector-delirium/log.log
2022-04-19 09:59:22,252 INFO cyclops.query.interface - Log file is /mnt/nfs/home/krishnanam/vector-delirium/log.log
2022-04-19 09:59:22,980 INFO cyclops.orm     - Database setup, ready to run queries!
2022-04-19 09:59:23,408 INFO cyclops.orm     - Query returned successfully!
2022-04-19 09:59:23,411 INFO cyclops.utils.profile - Finished executing function run_query in 0.409796 s
2022-04-19 09:59:25,907 INFO cyclops.orm     - Query return

## Process queried data into features. (Static aggregation + no imputation).

In [ ]:
import pandas as pd

from cyclops.processor import Aggregator, Imputer, featurize

diagnoses_data = pd.read_parquet("diagnoses.gzip")
vitals_data = pd.read_parquet("vitals.gzip")
labs_data = pd.read_parquet("labs.gzip")


# diagnoses_data = gather_columns(
#     diagnoses_data, [ENCOUNTER_ID, AGE, SEX, DIAGNOSIS_CODE]
# )
# feature_handler = featurize(
#     static_data=[diagnoses_data],
#     temporal_data=[labs_data, vitals_data],
#     imputer=Imputer(strategy="none"),
#     aggregator=Aggregator(strategy="static", range_=24, window=24),
# )

# sample_time = labs_data['lab_test_timestamp']
# admit_time = labs_data['admit_timestamp']

# sample_time = vitals_data['vital_measurement_timestamp']
# admit_time = vitals_data['admit_timestamp']
# diff = (sample_time - admit_time) / pd.Timedelta(hours=1)

## Inspect feature handler, and check features.

In [ ]:
print(feature_handler.get_numerical_features())
print(feature_handler.get_categorical_features())

feature_handler.features

## Look at data quality report (patient statics + labs + vitals)

In [ ]:
from evidently import ColumnMapping
from evidently.dashboard import Dashboard
from evidently.dashboard.tabs import DataQualityTab

dashboard = Dashboard(tabs=[DataQualityTab()])
column_mapping = ColumnMapping(
    numerical_features=feature_handler.get_numerical_features(),
    categorical_features=feature_handler.get_categorical_features(),
)
dashboard.calculate(
    reference_data=feature_handler.features,
    current_data=None,
    column_mapping=column_mapping,
)
dashboard.save(f"data_processed.html")